In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
    --------------------------------------- 0.3/14.3 MB 2.7 MB/s eta 0:00:06
   --- ------------------------------------ 1.2/14.3 MB 8.4 MB/s eta 0:00:02
   --- ------------------------------------ 1.3/14.3 MB 7.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.5/14.3 MB 6.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.7/14.3 MB 6.0 MB/s eta 0:00:03
   ----------- ---------------------------- 4.0/14.3 MB 12.2 MB/s eta 0:00:01
   -------------------- ------------------- 7.5/14.3 MB 20.0 MB/s eta 0:00:01
   --------------------- ------------------ 7.6/14.3 MB 17.9 MB/s eta 0:00:01
   --------------------- ------------------ 7.8/14.3 MB 16.6 MB/s eta 0:00:01
   ---------------------- ----------------- 8.2/14.3 MB 15.9 MB/s eta 0:00:01
   ----------------------- ---------------- 8.5/14.3 MB 16.0 MB/s eta 0:00:01
   ----

In [4]:
!pip install flask

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import mysql.connector

In [42]:
conn = mysql.connector.connect(
    host = "project-db-cgi.smhrd.com",
    port = 3307,
    user = "campus_24IS_IoT1_P2_2",
    password = "smhrd2",
    database = "campus_24IS_IoT1_P2_2"
    )
curs = conn.cursor()
# 누적 생산량 구하는 코드
#curs.execute("select * from tb_generated_power")
curs.execute("SELECT SUM(energy) FROM tb_generated_power WHERE DAY(created_at) >=1 and day(created_at) <= 31 ;")
#conn.commit()
st_prod = curs.fetchall()

In [24]:
type(st_prod[0][0])

decimal.Decimal

In [25]:
# 누적 축적량 구하는 코드
usage = 10000 *0.1
sql = f"""select sum(t1.energy) - {usage}
from (select distinct energy, created_at
		from tb_generated_power
		where created_at>= DATE_ADD(NOW(), INTERVAL -1 month)) t1;"""
curs.execute("SELECT SUM(energy) FROM tb_generated_power")
st_stack = curs.fetchall()
st_stack

[(Decimal('197.08'),)]

In [26]:
sql="""SELECT distinct avg(t1.efficienty)
FROM tb_generated_power t1
JOIN (
    SELECT panel_idx, MAX(created_at) AS latest
    FROM tb_generated_power
    GROUP BY panel_idx
) t2
ON t1.panel_idx = t2.panel_idx AND t1.created_at = t2.latest;"""
curs.execute(sql)
efficienty = curs.fetchall()

In [27]:
# 현재 평균 생산량 구하는 코드
sql = """SELECT distinct avg(t1.energy)
FROM tb_generated_power t1
JOIN (
    SELECT panel_idx, MAX(created_at) AS latest
    FROM tb_generated_power
    GROUP BY panel_idx
) t2
ON t1.panel_idx = t2.panel_idx AND t1.created_at = t2.latest;
"""
curs.execute(sql)
curr_prod = curs.fetchall()
curr_prod

[(Decimal('14.948571'),)]

In [28]:
# 배터리 충전 상태 구하는 코드
now = 0.5
usage = 10000 *0.1
battery = 10000
sql=f"""select ( {battery} * {now} + sum(t1.energy) - {usage}) /{battery} *100
from (select distinct energy , created_at from tb_generated_power) t1;
"""
curs.execute(sql)
charge = curs.fetchall()
charge

[(Decimal('41.848800'),)]

In [29]:
# 전압 구하는 코드
sql = """SELECT distinct avg(t1.votage)
FROM tb_generated_power t1
JOIN (
    SELECT panel_idx, MAX(created_at) AS latest
    FROM tb_generated_power
    GROUP BY panel_idx
) t2
ON t1.panel_idx = t2.panel_idx AND t1.created_at = t2.latest;"""
curs.execute(sql)
voltage =curs.fetchall()

In [30]:
print((float(round(voltage[0][0],2))))

2.42


In [31]:
### 주차장 이용 실태 칸 ###

# 이용률 높은 주차장 위치 코드
  # --> 예시값 넣을 예정 
# 평균 이용시간 코드
sql = """SELECT parking_lot, avg(parking_duration) as D
FROM tb_parking
where parking_lot like '자리%'
GROUP BY parking_lot 
order by D desc limit 1;"""
curs.execute(sql)
avg_duration =curs.fetchall()
# 전년대비 주차 이용률 코드
  # --> 예시값 넣을 예정 

In [32]:
print({ '이번달 누적 생산량' : st_prod, 
        '이번달 순수 생산량': st_stack, 
        "현재 생산 효율" : efficienty,
        '현재 생산량':curr_prod, 
        '배터리':charge, 
        '전압':voltage,
        "평균이용시간": avg_duration})
print("누적생산량",st_prod[0][0])
print("이번달 순수 생산량",st_stack[0][0])
print("현재 생산 효율", efficienty[0][0])
print("현재 생산량",curr_prod[0][0])
print("배터리",charge[0][0])
print("전압",voltage[0][0])
print("평균이용시간",avg_duration[0])



{'이번달 누적 생산량': [(Decimal('197.08'),)], '이번달 순수 생산량': [(Decimal('197.08'),)], '현재 생산 효율': [(Decimal('33.7143'),)], '현재 생산량': [(Decimal('14.948571'),)], '배터리': [(Decimal('41.848800'),)], '전압': [(Decimal('2.420000'),)], '평균이용시간': [('자리1', 0.5454545454545454)]}
누적생산량 197.08
이번달 순수 생산량 197.08
현재 생산 효율 33.7143
현재 생산량 14.948571
배터리 41.848800
전압 2.420000
평균이용시간 ('자리1', 0.5454545454545454)


In [43]:
sql="""SELECT distinct parking_lot 
        FROM tb_parking 
        where created_at>= date(date(now())-2) 
                        and(sensing_value1>170
                                or sensing_value1<2 
                                or sensing_value2>170 
                                or sensing_value2<2); """
curs.execute(sql)
test =curs.fetchall()
test

[('',), ('A0',)]

In [54]:
##초음파센서 이상치 탐지
ultraError = []
for i in range(len(test)):
    ultraError.append(test[i][0])

In [55]:
ultraError

['', 'A0']

In [57]:
sql="""SELECT distinct panel_idx 
        FROM tb_generated_power 
        where created_at>= date(date(now())-2)
            and energy<3;"""
curs.execute(sql)
test1 =curs.fetchall()

In [58]:
test1

[('101001',)]

In [59]:
panelError = []
for i in range(len(test1)):
    panelError.append(test1[i][0])

In [60]:
panelError

['101001']

In [89]:
data_lst = pd.DataFrame()

In [ ]:
file.to_csv('데이터.csv')